In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('PJT002_train.csv')
val = pd.read_csv('PJT002_validation.csv')
test = pd.read_csv('PJT002_test.csv')
sub = pd.read_csv('PJT002_submission.csv')
data = pd.concat([train, val])

In [44]:
df_Y = train[train['fr_yn'] == 'Y']
df_N = train[train['fr_yn'] == 'N']
df_N_sample = df_N.sample(frac = (len(df_Y)/len(df_N)) * (16/4)) #N과 Y 6대4샘플링
train = pd.concat([df_N_sample,df_Y])

In [45]:
train['dt_of_athrztn'].fillna(9999,inplace  = True)

In [46]:
train['dt_of_athrztn'] = train['dt_of_athrztn'].astype(str,inplace = True)

In [47]:
train['dt_of_athrztn']=train['dt_of_athrztn'].str[:4]

In [48]:
train['dt_of_athrztn'] = train['dt_of_athrztn'].astype(int,inplace = True)

In [52]:
train[train['dt_of_athrztn'] == 9999]['fr_yn'].unique()

array(['N'], dtype=object)

In [22]:
len(train['dt_of_athrztn'].unique())

172

In [23]:

bins = np.linspace(1600,2019, 20)
train['dt_of_athrztn'] = np.digitize(train['dt_of_athrztn'], bins)


In [54]:
val['dt_of_athrztn'].fillna(9999,inplace  = True)

In [56]:
test['dt_of_athrztn'].fillna(9999,inplace  = True)

In [57]:
test[test['dt_of_athrztn'] == 9999]['fr_yn'].unique()

array([nan])

In [55]:
val[val['dt_of_athrztn'] == 9999]['fr_yn'].unique()

array(['N'], dtype=object)

In [24]:
val['dt_of_athrztn'].fillna(9999,inplace  = True)

val['dt_of_athrztn'] = val['dt_of_athrztn'].astype(str,inplace = True)

val['dt_of_athrztn']=val['dt_of_athrztn'].str[:4]

val['dt_of_athrztn'] = val['dt_of_athrztn'].astype(int,inplace = True)

len(val['dt_of_athrztn'].unique())

bins = np.linspace(1600,2019, 20)
val['dt_of_athrztn'] = np.digitize(val['dt_of_athrztn'], bins)

test['dt_of_athrztn'].fillna(9999,inplace  = True)

test['dt_of_athrztn'] = test['dt_of_athrztn'].astype(str,inplace = True)

test['dt_of_athrztn']=test['dt_of_athrztn'].str[:4]

test['dt_of_athrztn'] = test['dt_of_athrztn'].astype(int,inplace = True)

len(test['dt_of_athrztn'].unique())
bins = np.linspace(1600,2019, 20)
test['dt_of_athrztn'] = np.digitize(test['dt_of_athrztn'], bins)

In [10]:
test = test.drop(['fr_yn'],1)

In [3]:
data['dt_of_fr'] = pd.to_datetime(data['dt_of_fr'])
train['dt_of_fr'] = pd.to_datetime(train['dt_of_fr'])
val['dt_of_fr'] = pd.to_datetime(val['dt_of_fr'])
test['dt_of_fr'] = pd.to_datetime(test['dt_of_fr'])



In [4]:
categorical_cols1 = train.select_dtypes(['object']).columns
for col in categorical_cols1:
    train[col] = pd.Categorical(train[col]).codes


train['DT_hour'] = (train['dt_of_fr'].dt.hour).astype(np.int8)
train['DT_day_week'] = (train['dt_of_fr'].dt.dayofweek).astype(np.int8)
train['DT_day_month'] = (train['dt_of_fr'].dt.day).astype(np.int8)
train['DT_M'] = (train['dt_of_fr'].dt.month).astype(np.int8)
train = train.drop('dt_of_fr', axis = 1)

In [7]:
train['train_yn'] = 0

In [9]:
train['train_yn'][train['prcpttn'] != 0] = 1

In [10]:
train.groupby('train_yn').sum() / train.groupby('train_yn').count()

,fr_yn,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,...,fr_fghtng_fclt_spcl_css_6_yn,us_yn,dngrs_thng_yn,slf_fr_brgd_yn,blk_dngrs_thng_mnfctr_yn,cltrl_hrtg_yn,DT_hour,DT_day_week,DT_day_month,DT_M
train_yn,,,,,,,,,,,,,,,,,,,,,
0,0.092293,5.751595,4.081492,3.402553,494.496536,1249.739304,3960.888739,2516.921453,2.817116,0.121573,...,-0.854197,-0.56701,-0.837506,-0.837997,-0.837506,-0.837997,11.838979,2.762396,16.386843,7.366716
1,0.130664,6.286694,4.367937,3.455757,638.518442,2521.726481,29892.556718,2576.632536,3.206713,0.132858,...,-0.857598,-0.55906,-0.835083,-0.835170,-0.835170,-0.835153,11.682044,3.000892,15.695305,6.439225


In [13]:
categorical_cols1 = val.select_dtypes(['object']).columns
for col in categorical_cols1:
    val[col] = pd.Categorical(val[col]).codes


val['DT_hour'] = (val['dt_of_fr'].dt.hour).astype(np.int8)
val['DT_day_week'] = (val['dt_of_fr'].dt.dayofweek).astype(np.int8)
val['DT_day_month'] = (val['dt_of_fr'].dt.day).astype(np.int8)
val['DT_M'] = (val['dt_of_fr'].dt.month).astype(np.int8)
val = val.drop('dt_of_fr', axis = 1)

In [14]:
categorical_cols1 = test.select_dtypes(['object']).columns
for col in categorical_cols1:
    test[col] = pd.Categorical(test[col]).codes


test['DT_hour'] = (test['dt_of_fr'].dt.hour).astype(np.int8)
test['DT_day_week'] = (test['dt_of_fr'].dt.dayofweek).astype(np.int8)
test['DT_day_month'] = (test['dt_of_fr'].dt.day).astype(np.int8)
test['DT_M'] = (test['dt_of_fr'].dt.month).astype(np.int8)
test = test.drop('dt_of_fr', axis = 1)

In [640]:
from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        bins = np.linspace(0, 1000000, 1000)
        train['gas_engry_us_201%s%s'%(i,j)] = np.digitize(train['gas_engry_us_201%s%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 1000)
        val['gas_engry_us_201%s%s'%(i,j)] = np.digitize(val['gas_engry_us_201%s%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 1000)
        test['gas_engry_us_201%s%s'%(i,j)] = np.digitize(test['gas_engry_us_201%s%s'%(i,j)], bins)
        

In [641]:
from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        bins = np.linspace(0, 1000000, 1000)
        train['gas_engry_us_201%s0%s'%(i,j)] = np.digitize(train['gas_engry_us_201%s0%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 1000)
        val['gas_engry_us_201%s0%s'%(i,j)] = np.digitize(val['gas_engry_us_201%s0%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 1000)
        test['gas_engry_us_201%s0%s'%(i,j)] = np.digitize(test['gas_engry_us_201%s0%s'%(i,j)], bins)

In [642]:
from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        bins = np.linspace(0, 1000000, 1000)
        train['ele_engry_us_201%s%s'%(i,j)] = np.digitize(train['ele_engry_us_201%s%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 1000)
        val['ele_engry_us_201%s%s'%(i,j)] = np.digitize(val['ele_engry_us_201%s%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 1000)
        test['ele_engry_us_201%s%s'%(i,j)] = np.digitize(test['ele_engry_us_201%s%s'%(i,j)], bins)
        

In [643]:
train['gassum'] = 0
val['gassum'] = 0
test['gassum'] = 0

In [644]:
from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        train['gassum'] = train['gassum'] + train['gas_engry_us_201%s%s'%(i,j)]
        val['gassum'] = val['gassum'] + val['gas_engry_us_201%s%s'%(i,j)]
        test['gassum'] = test['gassum'] + test['gas_engry_us_201%s%s'%(i,j)]
       
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        train['gassum'] = train['gassum'] + train['gas_engry_us_201%s0%s'%(i,j)]
        val['gassum'] = val['gassum'] + val['gas_engry_us_201%s0%s'%(i,j)]
        test['gassum'] = test['gassum'] + test['gas_engry_us_201%s0%s'%(i,j)]

In [645]:
from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        bins = np.linspace(0, 1000000, 1000)
        train['ele_engry_us_201%s0%s'%(i,j)] = np.digitize(train['ele_engry_us_201%s0%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 1000)
        val['ele_engry_us_201%s0%s'%(i,j)] = np.digitize(val['ele_engry_us_201%s0%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 1000)
        test['ele_engry_us_201%s0%s'%(i,j)] = np.digitize(test['ele_engry_us_201%s0%s'%(i,j)], bins)



In [646]:
train['elesum'] = 0
val['elesum'] = 0
test['elesum'] = 0

In [647]:
from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        train['elesum'] = train['elesum'] + train['ele_engry_us_201%s%s'%(i,j)]
        val['elesum'] = val['elesum'] + val['ele_engry_us_201%s%s'%(i,j)]
        test['elesum'] = test['elesum'] + test['ele_engry_us_201%s%s'%(i,j)]
       
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        train['elesum'] = train['elesum'] + train['ele_engry_us_201%s0%s'%(i,j)]
        val['elesum'] = val['elesum'] + val['ele_engry_us_201%s0%s'%(i,j)]
        test['elesum'] = test['elesum'] + test['ele_engry_us_201%s0%s'%(i,j)]

In [648]:
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):       
        train = train.drop(['ele_engry_us_201%s0%s'%(i,j)],1)
        train = train.drop(['gas_engry_us_201%s0%s'%(i,j)],1)


        val = val.drop(['ele_engry_us_201%s0%s'%(i,j)],1)
        val = val.drop(['gas_engry_us_201%s0%s'%(i,j)],1)


        test = test.drop(['ele_engry_us_201%s0%s'%(i,j)],1)
        test = test.drop(['gas_engry_us_201%s0%s'%(i,j)],1)
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):       
        train = train.drop(['ele_engry_us_201%s%s'%(i,j)],1)
        train = train.drop(['gas_engry_us_201%s%s'%(i,j)],1)


        val = val.drop(['ele_engry_us_201%s%s'%(i,j)],1)
        val = val.drop(['gas_engry_us_201%s%s'%(i,j)],1)


        test = test.drop(['ele_engry_us_201%s%s'%(i,j)],1)
        test = test.drop(['gas_engry_us_201%s%s'%(i,j)],1)

In [649]:
# train['q'] = 0
# train['q'] = train['elesum'] / train['ttl_ar']

In [650]:
# val['q'] = 0
# val['q'] = val['elesum'] / val['ttl_ar']

In [651]:
# test['q'] = 0
# test['q'] = test['elesum'] / test['ttl_ar']

In [652]:
# train['p'] = 0
# train['p'] = train['gassum'] / train['ttl_ar']

# val['p'] = 0
# val['p'] = val['gassum'] / val['ttl_ar']

# test['p'] = 0
# test['p'] = test['gassum'] / test['ttl_ar']

In [653]:
# train['t'] = 0
# train['t'] = train['elesum'] + train['gassum']
# val['t'] = 0
# val['t'] = val['elesum'] + val['gassum']
# test['t'] = 0
# test['t'] = test['elesum'] + test['gassum']

In [654]:
# train['tt'] = 0
# train['tt'] = train['t'] / train['ttl_ar']
# test['tt'] = 0
# test['tt'] = test['t'] / test['ttl_ar']
# val['tt'] = 0
# val['tt'] = val['t'] / val['ttl_ar']

In [15]:
bins = np.linspace(0,800000, 1000)
train['ttl_ar'] = np.digitize(train['ttl_ar'], bins)
bins = np.linspace(0,800000, 1000)
val['ttl_ar'] = np.digitize(val['ttl_ar'], bins)
bins = np.linspace(0,800000, 1000)
test['ttl_ar'] = np.digitize(test['ttl_ar'], bins)


In [ ]:
train

In [656]:
X_train = train.drop(['fr_yn'],axis = 1)
X_test = val.drop(['fr_yn'],axis = 1)
y_train = train['fr_yn']
y_test = val['fr_yn']
X = pd.concat([X_train,X_test])
y = pd.concat([y_train,y_test])

In [657]:

X_train.drop(['id'], axis = 1, inplace = True)
test.drop(['id'], axis = 1, inplace = True)
X_test.drop(['id'], axis = 1, inplace = True)

In [658]:
train['fr_yn'].sum()

7657

In [659]:
len(train)

38285

In [661]:
#모델 생성 및 학습
for i in range(300):
    
    import lightgbm as lgb
    Lgb = lgb.LGBMClassifier(n_estimators=i+1000, 
                             silent=False, 
                             random_state =10, 
                             max_depth=18,
                             num_leaves=128,
                             learning_rate = 0.1,
                             objective='binary',
                             metrics ='auc'
                            )
    model = Lgb.fit(X_train, y_train)

#test를 통한 모델 검증
    predictions = model.predict(X_test)
    proba = model.predict_proba(X_test)
    from sklearn.metrics import accuracy_score, roc_auc_score
    from sklearn.metrics import f1_score, recall_score
    score = roc_auc_score(y_test, predictions)
    score2 = accuracy_score(y_test, predictions)
    score3 = f1_score(y_test, predictions)
    score4 = recall_score(y_test, predictions)
#     print('roc_auc_score:',score)
#     print('accuracy_score:',score2)
    print(i+1,":",'F1', score3)
    #print('recall', score4)

1 : F1 0.49167145318782307
2 : F1 0.49123814995690895
3 : F1 0.49123814995690895
4 : F1 0.4913892078071182
5 : F1 0.4913892078071182
6 : F1 0.49154000573558937


KeyboardInterrupt: 

In [ ]:
predictions
#54
# roc_auc_score: 0.6914737170414987
# accuracy_score: 0.8153087851551174
#F1 0.5148

In [ ]:
proba

In [377]:
y = model.predict(test)
y_prob = model.predict_proba(test)


In [ ]:
y_prob

In [50]:
y

array([0, 1, 0, ..., 0, 0, 1], dtype=int8)

In [52]:
y_prob2

array([[9.99999938e-01, 6.17229811e-08],
       [7.35424038e-01, 2.64575962e-01],
       [9.99959758e-01, 4.02423327e-05],
       ...,
       [9.96727257e-01, 3.27274313e-03],
       [9.62001801e-01, 3.79981993e-02],
       [6.34868527e-02, 9.36513147e-01]])

In [47]:
y2 = model.predict(test)
y_prob2 = model.predict_proba(test)


In [68]:
a = pd.DataFrame((y_prob+ y_prob)/2)

In [69]:
a['tf'] = a[0] < a[1]

In [73]:
a['tf']

0       False
1        True
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15       True
16       True
17       True
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2927     True
2928    False
2929    False
2930    False
2931    False
2932     True
2933    False
2934    False
2935    False
2936    False
2937    False
2938    False
2939    False
2940    False
2941     True
2942    False
2943    False
2944    False
2945    False
2946    False
2947    False
2948     True
2949     True
2950    False
2951     True
2952    False
2953    False
2954    False
2955    False
2956     True
Name: tf, Length: 2957, dtype: bool

In [63]:
a['tf'] = a['tf'].map({False:'N',True:'Y'})

In [76]:
sub['fr_yn'] = a['tf']

In [378]:
sub['fr_yn'] = y
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.head()

,fr_yn
0,N
1,Y
2,N
3,N
4,N


In [379]:
sub.to_csv('tmddnjs94_화재예측과제_진우14파라미터.csv', index=False)

In [74]:
a['tf']
a['tf'] = pd.Categorical(a['tf']).codes

In [78]:
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})

In [85]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2957 entries, 0 to 2956
Data columns (total 1 columns):
fr_yn    2957 non-null object
dtypes: object(1)
memory usage: 23.2+ KB
